In [1]:
from __future__ import print_function

from keras.models import Sequential
from keras.layers import Dense, LSTM,Activation,Dropout

from keras.optimizers import RMSprop

import numpy as np
import random
import sys

Using TensorFlow backend.


In [4]:
path = 'shakespeare_final.txt'
text = open(path).read().lower()

print('corpus length:', len(text))
characters = sorted(list(set(text)))
print('total chars:', len(characters))


char2indices = dict((c, i) for i, c in enumerate(characters))
indices2char = dict((i, c) for i, c in enumerate(characters))

corpus length: 581432
total chars: 61


In [5]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 193798


In [6]:
# Converting indices into vectorized format
X = np.zeros((len(sentences), maxlen, len(characters)), dtype=np.bool)
y = np.zeros((len(sentences), len(characters)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char2indices[char]] = 1
    y[i, char2indices[next_chars[i]]] = 1

In [7]:
#Model Building
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(characters))))

model.add(Dense(len(characters)))

model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               97280     
_________________________________________________________________
dense_1 (Dense)              (None, 61)                7869      
_________________________________________________________________
activation_1 (Activation)    (None, 61)                0         
Total params: 105,149
Trainable params: 105,149
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
# Function to convert prediction into index
def pred_indices(preds, metric=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / metric
    exp_preds = np.exp(preds)
    preds = exp_preds/np.sum(exp_preds)
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)


In [9]:
# Train & Evaluate the Model

for iteration in range(1, 30):
    print('-' * 40)
    print('Iteration', iteration)
    model.fit(X, y,batch_size=128,epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.7,1.2]:

        print('\n----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(characters)))
            for t, char in enumerate(sentence):
                x[0, t, char2indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = pred_indices(preds, diversity)
            pred_char = indices2char[next_index]

            generated += pred_char
            sentence = sentence[1:] + pred_char

            sys.stdout.write(pred_char)
            sys.stdout.flush()
        print("\nOne combination completed \n")

----------------------------------------
Iteration 1
Epoch 1/1
193798/193798 [==============================] - 374s 2ms/step - loss: 1.9411

----- diversity: 0.2
----- Generating with seed: "tcheth not thy show,
    the soil is thi"
tcheth not thy show,
    the soil is think with with the come the counter with the seept the greet the graes
    and good with the suck and the grain
    and the greet with the grave the counter the come the greet the greet
    and good shall the greet with the come the come the come the seept the reat and greet be the come the come the grave the greet the come the will the dist with the come,
    and good with the rost with the read wi
One combination completed 


----- diversity: 0.7
----- Generating with seed: "tcheth not thy show,
    the soil is thi"
tcheth not thy show,
    the soil is this with him be chalotians
    more than commerd.
  siate to couth do butk and graes, it but preatert?
  is and there wis may fill the kent peas bent
    calsivert i 

  caesar. what i have not to the recivain to the mother.
    my a word, poor i could have true than honour, whose are upon the hand
    i be other world be the will'd with me of heart,
  that thou hast blood find at i know you.
  celia. that the eye it merrowious his did dementer will
    a body's night love hand be nother with the cause the marrail honour
    will biser as the court un could in 
One combination completed 


----- diversity: 1.2
----- Generating with seed: " nay, gentle madam, to him! comfort him."
 nay, gentle madam, to him! comfort him.
  enouath ungmist verse or kion?
  celia. have horse of in ilaurs on your fear her mideur; do more i count most lose. wilf we see, know's.


                      11ouni




lentent scenoan.
  us the dedrew count.
  erous fookisgd that dogett hims the got.
  soldier. what show tine part,
  to seouthity towery in most, alive andon me,
  and i macharm be
    the drust we most luon excure outsfolcac
One combination completed 

----------

    sleeve unbutton'd, yo"
unbanded, your
    sleeve unbutton'd, you with the strength thee,
    and be my love the strength and the strength that here the father that is the stranted the past,
    the strength the song the matter that the strength the strance,
    the streng the streng the man in the duke
    the streng. the seeks of the strength to the streng.
  countess. the duke the song the me the streng thee,
    the seelly the prove the part. the strance, 
One combination completed 


----- diversity: 0.7
----- Generating with seed: "unbanded, your
    sleeve unbutton'd, yo"
unbanded, your
    sleeve unbutton'd, you reasen, put thy at in nature, sir.
  cleolal. for proases, can drame theu, the most of love,
    for my purposes, to worts thou and is i have deceiv'd sorry
    more not i am not come is upon my been syractlew.
  king. but i frome device may be  but in thee with, whose comise. fintle foous
    let my love, to king the known from that heads, thy been to rosalindes.
  

 13312/193798 [=>............................] - ETA: 3:02 - loss: 2.7842hem hisbandles the what wisheruch a trog.
      oo the theld and the he breake her well iwble atrey; herpait what wild andondy
    igrent le tper and the whels my for me,
   
One combination completed 


----- diversity: 0.7
----- Generating with seed: "ian dish again; then shall the sighs of "
ian dish again; then shall the sighs of the theldg a therre comes
  my wid afher, and byg the wonle me beand, if you sint the and of hadon fear like her;
    i nouke the are my thantan. geat; marven thite
                 n~sine ither supies kist bet wilikin, tave men.
           yoy are firmm thiten me weand,
    a divers siply det beten thert in dial thite,
166912/193798 [========================>.....] - ETA: 27s - loss: 3.9972